In [ ]:
## -- Install paho-mqtt library using pip
## pip install paho-mqtt==1.6.1

"""Publish terminal messages to topic"""

import paho.mqtt.client as mqtt

# MQTT settings
broker_url = "3.138.185.79"
broker_port = 1883
username = "auca"
password = "gishushu"
topic = "auca_class"
client_id = "my_mqtt_client"

# Callback when the client receives a CONNACK response from the server
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected successfully to broker")
    else:
        print(f"Failed to connect, return code {rc}\n")
        # If the client fails to connect then we should stop the loop
        client.loop_stop()

# Create a new instance of the MQTT client with a specific client ID
client = mqtt.Client(client_id, clean_session=True)
client.on_connect = on_connect  # attach the callback function to the client
client.username_pw_set(username, password)  # set username and password

client.connect(broker_url, broker_port, 60)  # connect to the broker

# Start the network loop in a separate thread
client.loop_start()

try:
    while True:
        message = input("Enter message to publish or type 'exit' to quit: ")
        if message.lower() == 'exit':
            break
        client.publish(topic, message, qos=2)
except KeyboardInterrupt:
    print("Program interrupted by user, exiting...")

# Stop the network loop and disconnect
client.loop_stop()
client.disconnect()


Connected successfully to broker
Enter message to publish or type 'exit' to quit: ab
Enter message to publish or type 'exit' to quit: cd


In [ ]:
"""Subscribe to messages on topic"""


import paho.mqtt.client as mqtt

# MQTT settings
broker_url = "3.138.185.79"
broker_port = 1883
username = "auca"
password = "gishushu"
topic = "auca_class"
client_id = "my_mqtt_client_subscriber"

# Callback when the client receives a CONNACK response from the server
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected successfully to broker")
        # Subscribe to the topic once connected
        client.subscribe(topic, qos=2)
    else:
        print(f"Failed to connect, return code {rc}\n")

# Callback for when a PUBLISH message is received from the server
def on_message(client, userdata, msg):
    print(f"Message received on topic {msg.topic}: {msg.payload.decode()}")

# Create a new instance of the MQTT client with a specific client ID
client = mqtt.Client(client_id, clean_session=True)
client.on_connect = on_connect  # attach the connection callback function to the client
client.on_message = on_message  # attach the message callback function to the client

client.username_pw_set(username, password)  # set username and password
client.connect(broker_url, broker_port, 60)  # connect to the broker

# Start the network loop in a separate thread
client.loop_forever()


## Create a standard PostgreSQL table:

CREATE TABLE sensors (
   time        TIMESTAMPTZ       NOT NULL,
   location    TEXT              NOT NULL,
   device      TEXT              NOT NULL,
   voltage     DOUBLE PRECISION  NOT NULL,
   current     DOUBLE PRECISION  NOT NULL,
   frequency   DOUBLE PRECISION  NOT NULL,
   power       DOUBLE PRECISION  NOT NULL,
   energy      DOUBLE PRECISION  NOT NULL,
   PRIMARY KEY (time, location, device)
);


## Convert the table to a hypertable. Specify the name of the table you want to convert, and the column that holds its time values.

SELECT create_hypertable('sensors', 'time', chunk_time_interval => interval '15 minutes');


## Alter chunk_interval if need be: Impact on New Chunks Only
SELECT set_chunk_time_interval('sensors', INTERVAL '20 minutes');



## Insert Single Row

INSERT INTO sensor (time, location, device, voltage, current, frequency, power, energy) 
VALUES 
('2024-04-26 12:30:00+00', 'Factory A', 'Sensor 123', 230.5, 5.5, 60, 1267.5, 15000);


## Insert Multiple Rows

INSERT INTO sensors (time, location, device, voltage, current, frequency, power, energy) 
VALUES 
('2024-04-26 12:31:00+00', 'Factory A', 'Sensor 123', 230.5, 5.5, 60, 1267.5, 15000),
('2024-04-26 12:35:00+00', 'Factory B', 'Sensor 124', 220.0, 6.0, 50, 1320.0, 16000),
('2024-04-26 12:40:00+00', 'Factory C', 'Sensor 125', 240.0, 4.8, 50, 1152.0, 14000);

In [ ]:
## Enable Compression

ALTER TABLE sensor SET (timescaledb.compress, timescaledb.compress_orderby = 'time');

## Enable Compression; adding location and device can be advantageous for data retrieval and compression efficiency as this approach can help TimescaleDB more effectively compress data by grouping similar values together, which is especially useful when your queries often filter or join on these columns.

ALTER TABLE sensor SET (timescaledb.compress, timescaledb.compress_orderby = 'time', timescaledb.compress_segmentby = 'location, device');


## Add Compression Policy
SELECT add_compression_policy('sensor', INTERVAL '7 days');

## Show all chunks older than x minutes
SELECT show_chunks('sensor', older_than => INTERVAL '3 minutes');

## Show which chunks are available and their compression status:
SELECT chunk_name, range_start, range_end, is_compressed 
FROM timescaledb_information.chunks
WHERE hypertable_name = 'sensor';


## Compress all chunks older than 3 minutes
SELECT compress_chunk(i)
FROM show_chunks('sensor', older_than => INTERVAL '2 minutes') AS i;

## Drop Table
DROP TABLE TABLE_NAME;
DROP TABLE sensor;

## Enable Compression
ALTER TABLE sensor SET (timescaledb.compress, timescaledb.compress_orderby = 'time');

## Enable Compression; adding location and device can be advantageous for data retrieval and compression efficiency as this approach can help TimescaleDB more effectively compress data by grouping similar values together, which is especially useful when your queries often filter or join on these columns.

ALTER TABLE sensor SET (timescaledb.compress, timescaledb.compress_orderby = 'time', timescaledb.compress_segmentby = 'location, device');

## Add Compression Policy
SELECT add_compression_policy('sensor', INTERVAL '7 days');

## Show all chunks older than x minutes
SELECT show_chunks('sensors', older_than => INTERVAL '3 minutes');

## Compress all chunks older than 3 minutes¶
SELECT compress_chunk(i) FROM show_chunks('sensor', older_than => INTERVAL '3 minutes') AS i;

## Drop Table
DROP TABLE TABLE_NAME; 
DROP TABLE sensor;

## Disk Size of a hypertable; both compressed and uncompressed chunks
SELECT hypertable_size('sensor');

## Retrieves the name and size of each hypertable present in the database
SELECT 'sensors', hypertable_size(format('%I.%I', hypertable_schema, hypertable_name)::regclass) FROM timescaledb_information.hypertables;

## Detailed view of the disk space usage of a hypertable; If running on a distributed hypertable, ordering by node_name would show the size distribution across different data nodes 
SELECT * FROM hypertable_detailed_size('sensors') ORDER BY node_name;

## Manually compresses a specific chunk identified by its internal name.
SELECT compress_chunk( '_timescaledb_internal._hyper_1_1_chunk');

## Provides compression statistics for all chunks of the specified hypertable.
SELECT * FROM chunk_compression_stats('sensor');

## Attempts to compress chunks of the 'sensor' hypertable that were created between three weeks ago and one week ago.
SELECT compress_chunk(i) from show_chunks('sensor', now() - interval '1 week', now() - interval '3 weeks') i;


## Use the psycopg2 library to interact with your PostgreSQL (TimescaleDB) database

pip install psycopg2

In [ ]:
"""No Batch Inserts"""

import psycopg2
import random
from datetime import datetime, timedelta
import time  # Import the time module

# Database connection parameters
conn_params = {
    "dbname": 'postgres',
    "user": '',
    "password": '',
    "host": 'localhost'
}

# Function to generate random data, now with slightly adjusted time parameter
def generate_random_data(sensor_id, fixed_time, delta_seconds):
    time_adjusted = fixed_time + timedelta(seconds=delta_seconds)
    return {
        "time": time_adjusted,
        "location": f"Location {sensor_id}",
        "device": f"Sensor {sensor_id}",
        "voltage": random.uniform(220, 240),
        "current": random.uniform(10, 20),
        "frequency": random.uniform(50, 60),
        "power": random.uniform(1000, 5000),
        "energy": random.uniform(500, 1500)
    }

conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()

try:
    print("Data Insertion Started...")
    start_time = time.time()  # Capture start time
    fixed_time = datetime.now()
    for i in range(200000):
        sensor_id = random.randint(1, 10)
        data = generate_random_data(sensor_id, fixed_time, i * 0.001)  # Adjust time by 0.001 second increments

        query = """
        INSERT INTO sensor (time, location, device, voltage, current, frequency, power, energy) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """
        cursor.execute(query, (data['time'], data['location'], data['device'], data['voltage'],
                               data['current'], data['frequency'], data['power'], data['energy']))
        
        if i % 1000 == 0:
            conn.commit()
            ## print(f"Inserted {i+1} records")

    conn.commit()
    print("Data insertion complete.")

    end_time = time.time()  # Capture end time
    total_time = end_time - start_time  # Calculate duration
    print(f"Total time taken: {total_time:.2f} seconds.")

except Exception as e:
    print("An error occurred:", e)
    conn.rollback()

finally:
    cursor.close()
    conn.close()


## Use batch insertions for faster writes

In [ ]:
import psycopg2
import psycopg2.extras  # Import extras for execute_batch
import random
from datetime import datetime, timedelta
import time

# Database connection parameters
conn_params = {
    "dbname": 'postgres',
    "user": '',
    "password": '',
    "host": 'localhost'
}

# Function to generate random data, now with slightly adjusted time parameter
def generate_random_data(sensor_id, fixed_time, delta_seconds):
    time_adjusted = fixed_time + timedelta(seconds=delta_seconds)
    return (
        time_adjusted,
        f"Location {sensor_id}",
        f"Sensor {sensor_id}",
        random.uniform(220, 240),
        random.uniform(10, 20),
        random.uniform(50, 60),
        random.uniform(1000, 5000),
        random.uniform(500, 1500)
    )

# Connect to the PostgreSQL database
conn = psycopg2.connect(**conn_params)
cursor = conn.cursor()

try:
    print("Data Insertion Started...")
    start_time = time.time()  # Capture start time
    fixed_time = datetime.now()

    # Prepare data list for batch insert
    data_tuples = []
    for i in range(200000):
        sensor_id = random.randint(1, 10)
        data = generate_random_data(sensor_id, fixed_time, i * 0.001)  # Adjust time by 0.001 second increments
        data_tuples.append(data)

        # Execute batch every 1000 inserts
        if (i + 1) % 1000 == 0:
            psycopg2.extras.execute_batch(cursor, """
            INSERT INTO sensor (time, location, device, voltage, current, frequency, power, energy)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
            """, data_tuples)
            conn.commit()
            data_tuples = []  # Reset the list
            ## print(f"Inserted {i + 1} records")

    # Insert any remaining data
    if data_tuples:
        psycopg2.extras.execute_batch(cursor, """
        INSERT INTO sensor (time, location, device, voltage, current, frequency, power, energy)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """, data_tuples)
        conn.commit()

    print("Data insertion complete.")

    end_time = time.time()  # Capture end time
    total_time = end_time - start_time  # Calculate duration
    print(f"Total time taken: {total_time:.2f} seconds.")

except Exception as e:
    print("An error occurred:", e)
    conn.rollback()

finally:
    cursor.close()
    conn.close()



## SQL Analytics

In [ ]:
## -- Average, Max, and Min voltage readings for each device
SELECT device, AVG(voltage) AS avg_voltage, MAX(voltage) AS max_voltage, MIN(voltage) AS min_voltage
FROM sensor
GROUP BY device;


In [ ]:
## TimescaleDB provides a function called time_bucket() which is used to aggregate data over a 
## period of time. This is particularly useful for creating downsampled data views.
## -- Average current readings in 10-minute intervals for each device
SELECT time_bucket('10 minutes', time) AS period, device, AVG(current) AS avg_current
FROM sensor
GROUP BY period, device
ORDER BY period, device;


In [ ]:
## indexes that can optimize time-based lookups are not effective or absent
CREATE INDEX idx_sensor_device_time ON sensor(device, time);



## Python-SQL Analytics


In [ ]:
import psycopg2
import pandas as pd

# Establish connection
conn = psycopg2.connect(dbname='postgres', user='username', password='password', host='localhost')

# Query average daily energy consumption per device
query = """
SELECT time_bucket('1 day', time) AS day, device, AVG(energy) AS avg_daily_energy
FROM sensor
GROUP BY day, device
ORDER BY day, device;
"""

# Load query results into a DataFrame
df = pd.read_sql_query(query, conn)
print(df)


In [ ]:
## Compute basic statistics across all sensors to understand variations in energy use.

print(df.describe())


In [ ]:
## Check how many data points are available

print(df.shape)


In [ ]:
## Data Visualization: Plot energy readings across sensors to visualize differences.

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(df['device'], df['avg_daily_energy'])
plt.xlabel('Device')
plt.ylabel('Average Daily Energy')
plt.title('Energy Consumption by Sensor')
plt.xticks(rotation=45)
plt.show()


In [ ]:
## Clustering: Group sensors based on their energy consumption characteristics using 
## clustering algorithms. KMeans is a popular clustering algorithm used for partitioning data 
## into K distinct, non-overlapping clusters.

from sklearn.cluster import KMeans

# Assuming data normalization is appropriate here
kmeans = KMeans(n_clusters=3)
df['cluster'] = kmeans.fit_predict(df[['avg_daily_energy']])
print(df[['device', 'avg_daily_energy', 'cluster']])


In [ ]:
## Simple Comparisons and Correlations
## You can perform comparisons or check for correlations between sensors based on the available metrics.

# Example: Comparing average energy consumption
max_energy_device = df.loc[df['avg_daily_energy'].idxmax(), 'device']
print(f"Device with maximum average energy consumption: {max_energy_device}")

min_energy_device = df.loc[df['avg_daily_energy'].idxmin(), 'device']
print(f"Device with minimum average energy consumption: {min_energy_device}")
